<a href="https://colab.research.google.com/github/Aryanupadhyay23/Deep-Learning-/blob/main/multi_output_functional_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import kagglehub

path = kagglehub.dataset_download("jangedoo/utkface-new")
path

Using Colab cache for faster access to the 'utkface-new' dataset.


'/kaggle/input/utkface-new'

In [109]:
import shutil

zip_path = "/content/utkface-new.zip"
shutil.make_archive("/content/utkface-new", 'zip', path)

zip_path

'/content/utkface-new.zip'

In [110]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [131]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [132]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [133]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [134]:
len(age)

23708

In [135]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [136]:
df.shape

(23708, 3)

In [137]:
df.head()

,age,gender,img
0,70,0,70_0_3_20170119203826438.jpg.chip.jpg
1,25,0,25_0_1_20170113150701015.jpg.chip.jpg
2,29,1,29_1_4_20170104165735505.jpg.chip.jpg
3,23,0,23_0_1_20170117194052028.jpg.chip.jpg
4,26,0,26_0_3_20170119180330838.jpg.chip.jpg


In [138]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [139]:
train_df.shape

(20000, 3)

In [140]:
test_df.shape

(3708, 3)

In [141]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Config
IMG_SIZE = (200, 200)
BATCH_SIZE = 32
EPOCHS = 10

# For reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [142]:
# data generators and flow_from_dataframe (use class_mode='raw')
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=IMG_SIZE,
    class_mode='raw',    # return numpy array of shape (batch, 2)
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=IMG_SIZE,
    class_mode='raw',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [143]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [144]:
# build model (ResNet50 backbone + two heads)
resnet = ResNet50(include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), pooling=None)
resnet.trainable = False

# connect to last conv output and flatten
x = resnet.output
x = Flatten()(x)

# two separate branches
branch1 = Dense(512, activation='relu')(x)
branch2 = Dense(512, activation='relu')(x)

branch1 = Dense(512, activation='relu')(branch1)
branch2 = Dense(512, activation='relu')(branch2)

age_out = Dense(1, activation='linear', name='age')(branch1)
gender_out = Dense(1, activation='sigmoid', name='gender')(branch2)

model = Model(inputs=resnet.input, outputs=[age_out, gender_out])

model.compile(
    optimizer=Adam(),
    loss={'age': 'mae', 'gender': 'binary_crossentropy'},
    metrics={'age': 'mae', 'gender': 'accuracy'},
    loss_weights={'age': 1.0, 'gender': 99.0}
)

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 206, 206,  │          0 │ input_layer_10[0… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 100, 100,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 100, 100,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 100, 100,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 102, 102,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 50, 50,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 50, 50,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 50, 50,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 50, 50,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 50, 50,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 50, 50,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 50, 50,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 50, 50,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 50, 50,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 50, 50,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 50, 50,    │      1,024 │ conv2_block1_3_c

 Total params: 126,875,522 (483.99 MB)

 Trainable params: 103,287,810 (394.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [145]:
def wrapped_generator(generator):
    """Yields (images, {'age': age_batch, 'gender': gender_batch})"""
    for x_batch, y_batch in generator:
        # y_batch shape -> (batch, 2) where column0=age, column1=gender
        # Cast to correct dtype and shapes
        age_batch = y_batch[:, 0].astype('float32')        # shape (batch,)
        gender_batch = y_batch[:, 1].astype('float32')     # shape (batch,)
        yield x_batch, {'age': age_batch, 'gender': gender_batch}

In [146]:
history = model.fit(
    wrapped_generator(train_generator),
    epochs=EPOCHS,
    steps_per_epoch=50,
    validation_data=wrapped_generator(test_generator),
    validation_steps=10
)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 31s 415ms/step - age_loss: 23.7402 - age_mae: 23.7402 - gender_accuracy: 0.5138 - gender_loss: 6.1723 - loss: 634.7958 - val_age_loss: 16.0053 - val_age_mae: 16.0053 - val_gender_accuracy: 0.5188 - val_gender_loss: 0.7718 - val_loss: 92.4177
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 345ms/step - age_loss: 15.1867 - age_mae: 15.1867 - gender_accuracy: 0.4858 - gender_loss: 0.7458 - loss: 89.0221 - val_age_loss: 14.4001 - val_age_mae: 14.4001 - val_gender_accuracy: 0.5188 - val_gender_loss: 0.7112 - val_loss: 84.8073
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 325ms/step - age_loss: 15.9017 - age_mae: 15.9017 - gender_accuracy: 0.5077 - gender_loss: 0.7359 - loss: 88.7591 - val_age_loss: 14.9719 - val_age_mae: 14.9719 - val_gender_accuracy: 0.4969 - val_gender_loss: 0.7860 - val_loss: 92.7895
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 326ms/step - age_loss: 15.1591 - age_mae: 15.1591 - gender_accuracy: 0.5009 - gender_loss: 0.7382 - loss: 88.2360 - val_